# Can you read really fast?

#### Attention. Playing with the following script costs real money. Specifically, each request to OpenAI incurs a certain cost. Over two days of playing and testing, I spent \\$18 as part of the trial and then purchased access to ChatGPT Plus. I checked that summarizing a book by Jack London from 177,900 words to 1,782 (with creativity parameter 0.5) and to 1,807 words (with creativity parameter 0.0) costs \\$2.67.
#### Summary - it was supposed to be at the end, but I am providing it now because I value your time.

#### This was supposed to be a short and quick praise for how well OpenAI summarizes texts. During the few days of working on the script, I learned that:

##### * it's not advisable to summarize favorite books, as the result can be comical or pathetic.
##### * such summaries may be useful in business applications and can save a lot of time, especially when dealing with emails that are full of water or, worse, with friends who talk about nothing.


##### Often it's the case that we don't have the time or desire to read a long text. Or we'd like to quickly find out what the text is about and whether it's worth our most valuable resource, which is time. And that's when artificial intelligence comes to the rescue, which helps us perfectly with a quick summary of the text.
##### I have developed a system based on the ChatGPT language model, which summarizes long texts into shorter ones to save time.

In [ ]:
# Dependencies
import requests
import openai

##### I need a lengthy text. It would be great if this text is not only informative but also enjoyable to read and work with. I have found this kind of text.
##### I chose this link for two reasons. Firstly, it is textual - which makes it safer for testing. Secondly, I really like the concept of the Gutenberg project.
##### And in reality, I love this book.

In [ ]:
url = 'https://www.gutenberg.org/cache/epub/215/pg215.txt'

response = requests.get(url)

if response.status_code == 200:
    all_lengthy_text = response.content.decode('utf-8') # decode bytes to str    
else:
    print(f"Failed to retrieve content from {url}. Response code: {response.status_code}")

# I am cutting the text about the Gutenberg project license, it won't be necessary for this example.
all_lengthy_text = all_lengthy_text.split('*** START OF THE PROJECT GUTENBERG EBOOK THE CALL OF THE WILD ***')[1]
all_lengthy_text = all_lengthy_text.split('*** END OF THE PROJECT GUTENBERG EBOOK THE CALL OF THE WILD ***')[0]

# I am filtering empty lines
def filter_empty_lines(text):
    lines = text.split('\n')
    filtered_lines = [line for line in lines if line.strip()]
    filtered_text = '\n'.join(filtered_lines)
    return filtered_text

all_lengthy_text = filter_empty_lines(all_lengthy_text)

all_words = len(all_lengthy_text)

print(f'It is {len(all_lengthy_text)} words')

##### Time to launch the artificial intelligence. You will need your account on ChatGPT. Specifically, you will need your API key.

In [ ]:
%run key.ipynb # I have stored my API key in this Jupyter notebook, but I understand the importance of securing sensitive information and will take steps to protect it.

openai.api_key = api_key  # here enter your api key api_key = ""

basic_page_words = 2000
summary_factor = 0.3

##### Below is the definition of the summarizing function for a single page, which is based on the number of words in the page. The maximum context length of this model is 4097 tokens. In this example, I will use 2000 words and a summary factor of less than half for every page.

In [ ]:
def summarize_page(page):
    
    page_length = len(page)
    
    prompt = f"Please, summarizet his text for my 7 years old child : '{page}'"
    
    response = openai.Completion.create(
        engine = 'text-davinci-003',
        prompt = prompt,
        max_tokens=int(len(page)*summary_factor),
        n=1,    
        temperature=0.0
    )    
    
    summarize_page_length = len(response.choices[0].text)
    
    print(f'Page summarize facktor: {(summarize_page_length/page_length):.2f}\n')
    
    return response.choices[0].text

##### 	This is a quick test for the first page (more precisely, for the first 2000 words).

In [ ]:
quick_test = summarize_page(all_lengthy_text[0:basic_page_words])
quick_test = filter_empty_lines(quick_test)
print(f'{quick_test}')

##### The way this function below works is by dividing a long text into portions of 2000 words and requesting an AI summary for each portion. The summarized pages are then combined to create a new summarized full text. The function performs summarization only once.

In [ ]:
def summarize_once(text):
    
    all_words = len(text)
    
    start_word = 0
    end_word = start_word + basic_page_words
    
    summarize_text = ''
    
    while end_word < all_words:    
        
        page = text[start_word:end_word]
        
        summarize = summarize_page(page)
        summarize_text += summarize
        
        start_word += basic_page_words
        end_word = start_word + basic_page_words
        
    summarize_text_length = len(summarize_text)
    print(f'Real summarize factor is {(summarize_text_length/all_words):.2f}\n')
    
    return summarize_text

##### Finally, we continue to summarize the already summarized texts until we reach the desired size of the resulting text. For this example, I have set the summary length to one page, which is equivalent to 2000 words.

In [ ]:
summarized_text = summarize_once(all_lengthy_text)
summarized_text = filter_empty_lines(summarized_text)
length = len(summarized_text)

while length > 3 * basic_page_words:
    text = summarized_text
    summarized_text = summarize_once(text)
    summarized_text = filter_empty_lines(summarized_text)
    length = len(summarized_text)
    print(f'length: {length}')

summarized_text_length = len(summarized_text)
print(f'Summarized text with abstract ratio {(summarized_text_length/all_words):.2f}: \n\n{summarized_text}')

print(f'{len(summarized_text)}')

##### Do you really want to know what came out? Well, then download this Jupyter notebook and run it for your OpenAI API. It's really great fun, especially when you start experimenting with your favorite book.

##### Text summaries can be useful in various business areas such as:

##### * Project management: Summaries can help quickly review project documents and assess project progress.
##### * Human resources management: Summaries can help quickly understand resumes and cover letters of job candidates in the company.
##### * Competitive analysis: Summaries can help effectively and quickly review competitor reports, making it easier to make strategic decisions.
##### * Marketing: Summaries can help quickly review press articles, market reports, and analysis, making it easier to make marketing decisions.
##### * Legal activities: Summaries can help quickly understand legal documents and agreements, making it easier to make business and legal decisions.